# Customizing Lens

Lens strives to give you sensible defaults and automatically do the proper assessments whenever possible. However, there are many times where you'll want to change, select, or extend the functionality. Lens is intended to be an _extensible framework_ that can accomodate your own analysis plan.

In this document we will describe a number of ways you can customize Lens:
* Selecting which assessments to run
* Parameterizing assessments
* Creating new modules and assessments
* Incorporating new metrics

In [1]:
import credoai.lens as cl

## Setup

We'll set up data and a model to run through some customization options. See `quickstart` for more information on this code.

In [2]:
# imports for example data and model training
from credoai.data import fetch_creditdefault
from sklearn.ensemble import GradientBoostingClassifier
# data
data = fetch_creditdefault(as_frame=True)
X = data['data'].drop(columns=['SEX'])
y = data['target']
sensitive_feature = data['data']['SEX']
# model
model = GradientBoostingClassifier()
model.fit(X,y)

GradientBoostingClassifier()

In [3]:
credo_model = cl.CredoModel(name='credit_default_classifier',
                            model=model)

credo_data = cl.CredoData(name='UCI-credit-default',
                          X=X, 
                          y=y.astype(int),
                          sensitive_features=sensitive_feature)
alignment_manifest = {'metrics': ['precision_score']}

## Selecting Assessments

Lens has a number of assessments available, each of which works with different kinds of models or datasets. By default, Lens will automatically run every assessment that has its prerequesites met. The prerequesities can be queried by calling the `get_requirements` function on an assessment. These indicate the set of features or functions your model and data must instantiate in order for the assessment to be run. See `credoai.assessments.credo_assessment.AssessmentRequirements` for more.

In [4]:
from credoai.assessment.assessments import FairnessBaseAssessment
assessment = FairnessBaseAssessment()
assessment.get_requirements()

{'model_requirements': [('prob_fun', 'pred_fun')],
 'data_requirements': ['X', 'y', 'sensitive_features']}

But what if you don't want all of those assessments? No problem! Just pass the assessments you do want to Lens. 

In [5]:
# Note: Remember to instantiate the assessment!
selected_assessments = [FairnessBaseAssessment()]
lens = cl.Lens(model=credo_model,
               data=credo_data,
               assessments=selected_assessments,
               manifest=alignment_manifest)

## Parameterizing Assessments

Now that we can select assessments, how about customizing them? There are two places where assessments can be customized: (1) when their underlying module is initialized and (2) when they are ran (which runs the underlying module).

### Customizing initialization
Below we can see how to customize the initialization of the assessment. In this case we want to run an additional metric that isn't part of the `alignment manifest`. The parameters that can be passed at this stage are the same parameters passed to the assessment's `init_module` function.

In [11]:
# Note: Remember to instantiate the assessment!
selected_assessments = [FairnessBaseAssessment()]
init_kwargs = {'FairnessBase': {'additional_metrics': ['recall_score']}}
lens = cl.Lens(model=credo_model,
               data=credo_data,
               assessments=selected_assessments,
               manifest=alignment_manifest,
               init_assessment_kwargs=init_kwargs)

In [15]:
lens.run_assessments()['FairnessBase']['fairness']

,value
precision_score,0.000543
recall_score,0.01676


### Customizing running assessments 

The other way of parameterizing the assessments is by passing arguments to the assessment's `run` function. These kwargs are passed to `lens.run_assessments`, which are, in turn passed to the assessment's initialized module.

For instance, the `FairnessBase` assessment initializes `mod.FairnessBase`, whose `run` argument can take a `method` parameter which controls how fairness scores are calculated. The default is "between_groups", but we can change it like so:

In [16]:
run_kwargs = {'FairnessBase': {'method': 'to_overall'}}
lens.run_assessments(assessment_kwargs = run_kwargs)['FairnessBase']['fairness']

,value
precision_score,0.000302
recall_score,0.009504


## Creating New Modules & Assessments

## Custom Metrics

### incorporating confidence intervals

Custom metrics can be incorporated by creating a `FairnessFunction`. The `FairnessFunction` is a lightweight wrapper class that defines a few characteristics of the custom function needed by the module. 

**Example: Confidence Intervals**

Confidence intervals are not generically supported. However, they can be derived for metrics derived from a confusion matrix using the `wilson confidence interval`. A convenience function called `confusion_wilson` is supplied which returns an array: [lower, upper] bound for the metric. 

However, the module is not designed to accomodate metrics that return arrays. If you wish to incorporate CIs, we'd suggest creating a derived function that returns the lower bound, for example, and then wrapping it in a `FairnessFunction` as we mentioned for custom functions above. We show a case of that below:

In [ ]:
from credoai.utils.metrics import confusion_wilson
from credoai.assessment.model_assessments.fairness_base import FairnessFunction

# define partial functions for the true positive rate lower bound
def lower_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='tpr', confidence=0.95)[0]

# and upper bound
def upper_bound_tpr(y_true, y_pred):
    return confusion_wilson(y_true, y_pred, metric='tpr', confidence=0.95)[1]

# wrap the functions in fairness functions
lower_metric = FairnessFunction(name = 'lower_bound_tpr', 
                          func = lower_bound_tpr, 
                          takes_sensitive_attributes = False, 
                          takes_prob = False)

upper_metric = FairnessFunction(name = 'upper_bound_tpr', 
                          func = upper_bound_tpr, 
                          takes_sensitive_attributes = False, 
                          takes_prob = False)
           
# then proceed as normally
custom_assessment = FairnessModule([lower_metric, 
                           'true_positive_rate', 
                           upper_metric],
                A_str_test,
                Y_test,
                test_preds,
                test_scores,
                fairness_bounds
               )

custom_assessment.get_disaggregated_results(calculate_risk=False)